In [1]:
from glob import glob
import os   
import sys
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import pydicom

In [ ]:
def dcm_to_png(dcm_file_path, output_folder):
    # Read the DICOM file
    dicom_image = pydicom.dcmread(dcm_file_path)

    # Convert to a numpy array and normalize pixel values
    image_data = dicom_image.pixel_array
    image_data = (image_data - image_data.min()) / (image_data.max() - image_data.min()) * 255
    image_data = image_data.astype(np.uint8)

    # Convert to PIL Image
    img = Image.fromarray(image_data)

    # Save as PNG
    png_filename = os.path.splitext(os.path.basename(dcm_file_path))[0] + '.png'
    output_path = os.path.join(output_folder, png_filename)
    img.save(output_path)
    print(f"Saved {output_path}")